# Perceptró aplicat a iris

**Lectura del corpus:** $\;$ comprovem també que les matrius de dades i etiquetes tenes les files i columnes que toca

In [ ]:
# Importar la biblioteca numpy i assignar-li l'àlies 'np' per facilitar el seu ús posterior.
import numpy as np

# Importar la funció 'load_iris' del mòdul 'datasets' de la biblioteca 'sklearn'.
from sklearn.datasets import load_iris

# Carregar el conjunt de dades Iris i emmagatzemar-lo en la variable 'iris'.
iris = load_iris()

# Accedir a les dades de característiques (features) del conjunt de dades Iris, i convertir-les a 'float16'
X = iris.data.astype(np.float16)

# Accedir a les etiquetes (objectius), convertir-les al tipus de dada 'uint' i remodelar-les perquè siguen una matriu d'una columna
y = iris.target.astype(np.uint).reshape(-1, 1)

# Imprimir la forma (dimensions) de les matrius 'X' i 'y', seguit de les primeres 5 files
print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :])

**Partició del corpus:** $\;$ Creem un split d'iris amb un $20\%$ de dades per a test i la resta per a entrenament (training), barallant prèviament les dades d'acord amb una llavor donada per a la generació de nombres aleatoris. Ací, com en tot codi que incloga aleatorietat (que requerisca generar nombres aleatoris), convé fixar la dita llavor per a poder reproduir experiments amb exactitud.

In [ ]:
# Importar la funció 'train_test_split' del mòdul 'model_selection' de la biblioteca 'sklearn'.
from sklearn.model_selection import train_test_split

# Dividir els conjunts de dades de característiques (X) i etiquetes (y) en subconjunts d'entrenament i prova.
# El paràmetre 'test_size=0.2' indica que el 20% de les dades s'utilitzarà com a conjunt de prova, el 80% restant serà el conjunt d'entrenament.
# El paràmetre 'shuffle=True' assegura que les dades es barregen abans de dividir-les, la qual cosa és important per evitar sesgos.
# El paràmetre 'random_state=23' s'utilitza per garantir la reproductibilitat del procés de divisió.
# Els conjunts d'entrenament i prova resultants s'emmagatzemen en 'X_train', 'X_test', 'y_train' i 'y_test', respectivament.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)

# Imprimir les dimensions dels conjunts d'entrenament i prova de les característiques (X), mostrant informació sobre quantes mostres hi ha en cada conjunt.
print(X_train.shape, X_test.shape)

**Implementació de Perceptró:** $\;$ torna pesos en notació homogènia, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$ també el nombre d'errors i iteracions executades

In [ ]:
def perceptron(X, y, b=0.1, a=1.0, K=200):
    # X: Matriu de característiques d'entrada.
    # y: Vector d'etiquetes.
    # b: Marge de classificació.
    # a: Taxa d'aprenentatge.
    # K: Número màxim d'iteracions.

    N, D = X.shape  # N: Nombre de mostres, D: Nombre de característiques.
    Y = np.unique(y)  # Y: Etiquetes úniques en el conjunt de dades.
    C = Y.size  # C: Nombre de classes úniques.
    W = np.zeros((1+D, C))  # W: Matriu de pesos inicialitzada a zeros.

    for k in range(1, K+1):  # Bucle sobre les iteracions.
        E = 0  # E: Comptador d'errors en la classificació.

        for n in range(N):  # Bucle sobre totes les mostres.
            xn = np.array([1, *X[n, :]])  # xn: Vector de característiques amb un terme independent afegit.
            cn = np.squeeze(np.where(Y==y[n]))  # cn: Índex de la classe actual de la mostra.
            gn = W[:,cn].T @ xn  # gn: Producte punt de la mostra amb els pesos de la seua classe.
            err = False  # err: Indicador d'error en la classificació.

            # Bucle per actualitzar els pesos si es troba un error.
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn  # Actualitzar pesos per classes incorrectes.
                    err = True

            # Actualitzar pesos per la classe correcta si es va trobar un error.
            if err:
                W[:, cn] = W[:, cn] + a*xn
                E = E + 1  # Incrementar el comptador d'errors.

        # Si no hi ha errors, aturar l'entrenament.
        if E == 0:
            break;

    return W, E, k  # Tornar la matriu de pesos, el nombre d'errors i el nombre d'iteracions realitzades.

**Aprenentatge d'un classificador (lineal) amb Perceptró:** $\;$ Perceptró minimitza el nombre d'errors d'entrenament (amb marge)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{I}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{x}_n\biggr)$$

In [ ]:
W, E, k = perceptron(X_train, y_train)
print("Nombre d'iteracions executades: ", k)
print("Nombre d'errors d'entrenament: ", E)
print("Vectors de pesos de les classes (en columnes i amb notació homogènia):\n", W);

**Càlcul de la taxa d'error en test:**

In [ ]:
# Preparar el conjunt de dades de prova (X_test) per a la classificació.
# Això es fa afegint una columna d'uns a l'inici de X_test.
# La columna d'uns actua com el terme independent en la normalització per al model de perceptró.
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])

# Realitzar la predicció en el conjunt de prova.
# Això es fa calculant el producte (usant l'operador '@') entre X_testh i la matriu de pesos W.
# La funció 'np.argmax' s'utilitza per seleccionar l'índex de la classe amb el valor més alt en cada fila, el que correspon a la classe predita pel model.
# Els resultats es reformen en una matriu d'una columna.
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)

# Calcular la taxa d'error de les prediccions.
# Això es fa comptant el nombre de vegades que les prediccions (y_test_pred) difereixen de les etiquetes veritables (y_test),
# i dividint aquest nombre per la longitud del conjunt de prova (X_test).
# El resultat és la proporció de prediccions incorrectes.
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)

# Imprimir la taxa d'error en el conjunt de prova.
# El resultat es mostra com un percentatge amb un decimal.
print(f"Taxa d'error en test: {err_test:.1%}")

**Ajust del marge:** $\;$ experiment per a aprendre un valor de $b$

In [ ]:
# Bucle sobre diferents valors del paràmetre de marge 'b'.
for b in (.0, .01, .1, 10, 100):
    # Entrenar el perceptró amb el conjunt d'entrenament (X_train, y_train), utilitzant el valor actual de 'b' i un límit màxim de 1000 iteracions.
    # La funció 'perceptron' retorna la matriu de pesos 'W', el nombre d'errors 'E' i el nombre d'iteracions 'k'.
    W, E, k = perceptron(X_train, y_train, b=b, K=1000)

    # Imprimir el valor actual de 'b', el nombre d'errors 'E', i el nombre d'iteracions 'k' per a cada entrenament.
    print(b, E, k)

**Interpretació de resultats:** $\;$ les dades d'entrenament no semblen linealment separables; no està clar que un marge major que zero puga millorar resultats, sobretot perquè sols tenim $30$ mostres de test; amb marge nul ja hem vist que s'obté un error (en test) del $16.7\%$

In [ ]:
for b in (.0, .01, .1, 10, 100):

    W, E, k = perceptron(X_train, y_train, b=b, K=200)

    X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])

    y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)

    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)

    print(f"Iteració amb b: {b} Taxa d'error en test: {err_test:.1%}")

In [ ]:
for a in ( .01, .1, 10, 100):

    W, E, k = perceptron(X_train, y_train, a=a, K=1000)

    X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])

    y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)

    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)

    print(f"Iteració amb a: {a} Taxa d'error en test: {err_test:.1%}")

In [ ]:
for k in ( 200, 500, 1000, 10000):

    W, E, k = perceptron(X_train, y_train, a=0.1, b=10, K=k)

    X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])

    y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)

    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)

    print(f"Iteració amb k: {k} Taxa d'error en test: {err_test:.1%}")

Quina combinació de valors de a, b i k seleccionaries? Justifica la resposta.